# Elasticsearch playground

Jupyther notebook to play with Elasticsearch in general.
Connect to the cluster, play with queries, play with LLMChains.



In [1]:
import os
from langchain import LLMChain, OpenAI, Cohere, HuggingFaceHub, PromptTemplate
#from langchain.model_laboratory import ModelLaboratory

from elasticsearch import Elasticsearch

    

## Cluster credentials and URL

In [2]:

ES_URL = os.environ.get("ES_URL", "https://test1.es.europe-north1.gcp.elastic-cloud.com:443")
KB_URL = os.environ.get("KB_URL", "https://test1.kb.europe-north1.gcp.elastic-cloud.com:9243")

username = "elastic"
password = os.environ.get("ES_PASSWORD", "")

# Connect to the cluster in Elastic Cloud

In [3]:
client = Elasticsearch(ES_URL,
                       basic_auth=(username, password),
                       http_compress=True
                       )

client.info()

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '24ee0a66ae1544e3958cd6069dd0e626', 'cluster_uuid': 'rCAnFuk1QZ27xyXaBlvouQ', 'version': {'number': '8.7.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f229ed3f893a515d590d0f39b05f68913e2d9b53', 'build_date': '2023-04-27T04:33:42.127815583Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Use Elasticsearch API

In [13]:
my_index = "test-123"

index_settings = { "number_of_shards": 1 }
index_mappings = {
        "properties": {
            "name": { "type": "text"},
            "foo": { "type": "keyword" }
            }
}

#client.options(ignore_status=400).indices.delete(index=my_index)
client.options(ignore_status=400).indices.create(index=my_index, mappings=index_mappings, settings=index_settings)

# client.indices.refresh(index=my_index)

client.count(index="*")


ObjectApiResponse({'count': 31808, '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0}})

In [14]:
client.indices.get_mapping(index=my_index)


ObjectApiResponse({'test-123': {'mappings': {'properties': {'foo': {'type': 'keyword'}, 'name': {'type': 'text'}}}}})

In [18]:

client.indices.stats(index=my_index)


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 0, 'deleted': 0}, 'shard_stats': {'total_count': 1}, 'store': {'size_in_bytes': 225, 'total_data_set_size_in_bytes': 225, 'reserved_in_bytes': 0}, 'indexing': {'index_total': 0, 'index_time_in_millis': 0, 'index_current': 0, 'index_failed': 0, 'delete_total': 0, 'delete_time_in_millis': 0, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0, 'write_load': 0.0}, 'get': {'total': 0, 'time_in_millis': 0, 'exists_total': 0, 'exists_time_in_millis': 0, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 3, 'query_time_in_millis': 0, 'query_current': 0, 'fetch_total': 0, 'fetch_time_in_millis': 0, 'fetch_current': 0, 'scroll_total': 0, 'scroll_time_in_millis': 0, 'scroll_current': 0, 'suggest_total': 0, 'suggest_time_in_millis': 0, 'suggest_current': 0}, 'merges': {'current': 0

In [19]:

aliases = client.indices.get_alias(index="*", ignore_unavailable=True, )

aliases.keys()

/tmp/ipykernel_120307/147764811.py:1: ElasticsearchWarning: this request accesses system indices: [.apm-agent-configuration, .kibana_security_session_1, .security-tokens-7, .security-7, .apm-custom-link, .security-profile-8, .async-search, .kibana_8.7.1_001, .kibana_task_manager_8.7.1_001], but in a future major version, direct access to system indices will be prevented by default
  aliases = client.indices.get_alias(index="*", ignore_unavailable=True, )


dict_keys(['.kibana-event-log-8.7.1-000001', 'test-123', '.fleet-file-data-agent-000001', '.apm-agent-configuration', '.apm-source-map', '.kibana_security_session_1', '.fleet-files-agent-000001', 'kibana_sample_data_flights', '.security-tokens-7', '.security-7', '.apm-custom-link', 'kibana_sample_data_ecommerce', '.security-profile-8', '.async-search', '.kibana_8.7.1_001', '.kibana_task_manager_8.7.1_001'])

In [ ]:
datastreams = client.indices.get_data_stream(name="*")

datastreams.get("data_streams")

streams = client.indices.get_data_stream(name="*").get("data_streams")
streams
mylist = map(lambda info: info['name'], streams)
list(mylist)

['kibana_sample_data_logs']

# From here I test Elasticsearch Agent

In [1]:
import os
#os.environ["LANGCHAIN_HANDLER"] = "langchain"
#os.environ["LANGCHAIN_TRACING"] = "true"
#os.environ["LANGCHAIN_SESSION"] = "my_session" # Make sure this session actually exists. You can create a new session in the UI.

from langchain import OpenAI, ESEngine, ESChain
from langchain.chat_models import ChatOpenAI

ES_URL = os.environ.get("ES_URL", "https://test1.es.europe-north1.gcp.elastic-cloud.com:443")
KB_URL = os.environ.get("KB_URL", "https://test1.kb.europe-north1.gcp.elastic-cloud.com:9243")

username = "elastic"
password = os.environ.get("ES_PASSWORD", "")

db = ESEngine.from_uri(ES_URL, 
                       connection_args={
                           "basic_auth": (username, password),
                           "http_compress": True,
                       },
                       sample_docs_in_index_info = 0,
                       )


#db.get_usable_index_names()
#db._get_datastream_names()
#db.get_index_fields(index_name="kibana_sample_data_ecommerce")
#db._get_sample_docs(index_name="kibana_sample_data_ecommerce")
#print(db.get_index_info(index_names=["kibana_sample_data_ecommerce"]))
#print(db.get_index_info())

In [2]:

from langchain.agents import create_elasticsearch_agent
from langchain.agents.agent_toolkits import ElasticsearchDatastoreToolkit
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.es_engine import ESEngine
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

#db = ESEngine.from_uri("sqlite:///../../../../notebooks/Chinook.db")
llm = OpenAI(temperature=0, verbose=True)
toolkit = ElasticsearchDatastoreToolkit(db=db, llm=llm)

agent_executor = create_elasticsearch_agent(
    llm=OpenAI(temperature=0, verbose=True),
    toolkit=toolkit,
    top_k=0,
    verbose=True
)


In [3]:
#agent_executor.run("Describe ecommerce index.")

In [4]:

#agent_executor.run("Who bought the most in e-commerce shop?")
#agent_executor.run("What's the total revenue of my ecommerce shop?")
agent_executor.run("What's median spending in the last 7 days in my ecommerce shop?")
#agent_executor.run("Flights to which destination city are the most delayed?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "list_indices_es_db",
  "action_input": ""
}
```


Observation: test-123, kibana_sample_data_ecommerce, kibana_sample_data_logs, kibana_sample_data_flights, .fleet-files-agent-000001, .fleet-file-data-agent-000001
Thought: I should query the kibana_sample_data_ecommerce index to get the median spending in the last 7 days.
Action:
```
{
  "action": "query_es_db",
  "action_input": {
    "index_name": "kibana_sample_data_ecommerce",
    "query": {
      "size": 0,
      "aggs": {
        "median_spending": {
          "date_histogram": {
            "field": "order_date",
            "fixed_interval": "7d",
            "min_doc_count": 0
          },
          "aggs": {
            "median_spending": {
              "percentiles": {
                "field": "taxful_total_price",
                "percents": [50]
              }
            }
          }
        }
      }
    }
  }
}
```


Observation: {'took': 7, 'timed_out

'The median spending in the last 7 days in your ecommerce shop is 67.0'

In [17]:
print(db.get_index_info(index_names=["kibana_sample_data_ecommerce"]))

Index: kibana_sample_data_ecommerce
Fields: {'category': 'text', 'category.keyword': 'keyword', 'currency': 'keyword', 'customer_birth_date': 'date', 'customer_first_name': 'text', 'customer_first_name.keyword': 'keyword', 'customer_full_name': 'text', 'customer_full_name.keyword': 'keyword', 'customer_gender': 'keyword', 'customer_id': 'keyword', 'customer_last_name': 'text', 'customer_last_name.keyword': 'keyword', 'customer_phone': 'keyword', 'day_of_week': 'keyword', 'day_of_week_i': 'integer', 'email': 'keyword', 'event.dataset': 'keyword', 'geoip.city_name': 'keyword', 'geoip.continent_name': 'keyword', 'geoip.country_iso_code': 'keyword', 'geoip.location': 'geo_point', 'geoip.region_name': 'keyword', 'manufacturer': 'text', 'manufacturer.keyword': 'keyword', 'order_date': 'date', 'order_id': 'keyword', 'products._id': 'text', 'products._id.keyword': 'keyword', 'products.base_price': 'half_float', 'products.base_unit_price': 'half_float', 'products.category': 'text', 'products.ca

# END OF MY TEST. Following code to be reused later